In [0]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [0]:
grouped_purchased = pd.read_csv('cvs_data_file/online-retail.csv', header = 0)
grouped_purchased.head()

In [0]:
customers = list(np.sort(grouped_purchased.CustomerID.unique())) # Get our unique customers
products = list(grouped_purchased.StockCode.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.Quantity) # All of our purchases

rows = grouped_purchased.CustomerID.astype('category', categories = customers).cat.codes 
# Get the associated row indices
cols = grouped_purchased.StockCode.astype('category', categories = products).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

In [0]:
print(purchases_sparse.A)

In [0]:
import random

In [0]:
def make_train(ratings, pct_test = 0.2):
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered

In [0]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)


In [0]:
item_lookup = pd.read_csv('cvs_data_file/item_lookup.csv', header = 0)
item_lookup.head()

In [0]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return item_lookup.loc[item_lookup.StockCode.isin(prod_codes)]

In [0]:
import pickle

with open('saved_model','rb') as f:
            saved_model=pickle.load(f)
            model=saved_model['model']

In [0]:
# item TO item recommendation
from sklearn.metrics.pairwise import cosine_similarity

def display_item_to_items_recommendations(model,item_id):

    products_arr = np.array(products) 
    
#     print("products_arr",products_arr,str(item_id))
    item_id = np.where(products_arr == item_id)[0][0]
    print(item_id)
  
    return item_lookup['Description'][cosine_similarity(
            model.item_embeddings)[item_id].argsort()][-5:][::-1]


display_item_to_items_recommendations(model, str('85123A'))


In [0]:
def display_recommended_items(model, data, user_ids):
    customers_arr = np.array(customers)
    products_arr = np.array(products) 
    print("customers_arr",customers_arr)
    user_id = np.where(customers_arr == user_ids)[0][0]
#     print(user_ids,user_id)
    
    n_users, n_items = data.shape
#     known_positives = get_items_purchased(user_ids, product_train, customers_arr, products_arr, item_lookup)
    known_positives = item_lookup['Description'][data.tocsr()[user_id].indices]
    known_positives_df = pd.DataFrame(data=known_positives)
    print(known_positives_df,'\n','######################################')            
    
    
    scores = model.predict(user_id, np.arange(n_items))

#     print([np.argsort(-scores)])
    
    top_items = item_lookup['Description'][np.argsort(-scores)]
    print(top_items)
    
#     StockCode_id = str(item_lookup['StockCode'][purchases_sparse.tocsr()[user_id].indices][0])
#     print("StockCode_id",StockCode_id)
    
#     print('\n@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n',display_item_to_items_recommendations(model, StockCode_id))
            

display_recommended_items(model, product_train, 18282)